In [ ]:
# !pip install shap
# !pip install category_encoders

In [ ]:
import numpy as np
import shap
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import cohen_kappa_score
from sklearn.impute import KNNImputer
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
import re
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import seaborn as sn 
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, make_scorer, accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import category_encoders as ce

pd.set_option('max_rows', 1000)
pd.set_option('max_columns', 1000)
print("Numpy version {}".format(np.__version__))
print("Pandas version {}".format(pd.__version__))

In [ ]:
cluster_set = pd.read_csv("/content/Conflict + Food Insecurity.csv")

In [ ]:
cluster_set.columns

In [ ]:
# The target variable from clustering
cluster_set['clusteringlabels'].isnull().sum(axis=0)

In [ ]:
def sklearn_qwk(y_true, y_pred) -> np.float64:
    """
    Function for measuring Quadratic Weighted Kappa with scikit-learn
    
    :param y_true: The ground truth labels
    :param y_pred: The predicted labels
    
    :return The Quadratic Weighted Kappa Score (QWK)
    """
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")

In [ ]:
# path to data file
path = "/content/Dataset_2020_22.csv"

df = pd.read_csv(path, encoding='ISO-8859-1')

In [ ]:
# The cluster labels are in the same order as the dataset is
# therefore, we can directly get the target column from the cluster dataset
df['target'] = cluster_set['clusteringlabels']

In [ ]:
df.columns

In [ ]:
df.replace("à", 'a', regex=True, inplace=True)
df.replace("à", 'a', regex=True, inplace=True)
df.replace("è", 'e', regex=True, inplace=True)
df.replace("é", 'e', regex=True, inplace=True)
df.replace("ï", 'i', regex=True, inplace=True)
df[df['admin1']=='menaka']

In [ ]:
df.head()

In [ ]:
# removing all the burden data
to_drop = ['GAM_Burden_R_rest', 'SAM_Burden_R_rest', 'MAM_Burden_R_rest', 'gam_burden', 'sam_burden', 'mam_burden']

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.drop(to_drop, axis=1, inplace=True)

In [ ]:
df.dtypes

In [ ]:
df.describe(include=[object])

In [ ]:
# checking the columns having object data type 'x'
df['Food_Insecurity_Probability'].values

def is_float(x):
    try:
        float(x)
        return True
    except ValueError:
        return False


print([x for x in df['Food_Insecurity_Probability'].values if is_float(x) == False])
print([x for x in df['Food_Security'].values if is_float(x) == False])
print([x for x in df['INFORM_Mortality_rate_under-5'].values if is_float(x) == False])

In [ ]:
# replacing 'x' by np.NaN
df.loc[df['Food_Insecurity_Probability'] == 'x', 'Food_Insecurity_Probability'] = np.NaN
df.loc[df['Food_Security'] == 'x', 'Food_Security'] = np.NaN
df.loc[df['INFORM_Mortality_rate_under-5'] == 'x', 'INFORM_Mortality_rate_under-5'] = np.NaN

df['Food_Insecurity_Probability'] = df['Food_Insecurity_Probability'].astype(float)
df['Food_Security'] = df['Food_Security'].astype(float)
df['INFORM_Mortality_rate_under-5'] = df['INFORM_Mortality_rate_under-5'].astype(float)

In [ ]:
# checking integer columns
df.describe(include=['number'])

In [ ]:
# comparing the 50/ 75/ max values in the above table

cols_for_transformation = ['population_totale', 'population_6-59_month',	'mam_6-59_mois_prevalence', 'idps', 'Population_sans_acces_aux_structures_de_sante']

for col in cols_for_transformation:
  df[col] = np.log(df[col]+1)

In [ ]:
# dropping duplicates in 2021
df['key'] = df["country"].astype(str) + df['admin1'].astype(str) + df['ADMIN_2_Admnistratif'].astype(str) + df['admin2_sanitary'].astype(str)
df = df.loc[~(df.index.isin([783,781]))]
print(df.shape)
df['key'].value_counts()

In [ ]:
#Intersection
df = df[df['key'].map(df['key'].value_counts()) >= 3]
print(df.shape)

In [ ]:
df.drop(['key'], axis=1, inplace=True)
df.shape

In [ ]:
# removing the columns having more than 70% missing values
print(df.shape)
df.drop(df.columns[df.isnull().sum(axis=0)/ df.shape[0] > 0.7], axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
df.columns

In [ ]:
target_column = 'target'

In [ ]:
# target column has 14 missing values out of total 1102 values. Checking year wise split and removing those 14 rows
df['target_null'] = df[target_column].isnull()

print(df.groupby('year')['target_null'].count())

print(df.groupby('year')['target_null'].sum())


In [ ]:
df = df.loc[df[target_column].isnull()==False, :]
df.reset_index(drop=True, inplace=True)

df.drop('target_null', axis=1, inplace=True)

In [ ]:
# seeing region wise distribution in different years
area_distribution = df.groupby(['year', 'admin1'])['priority_level_validated_by_the_clusters'].count().reset_index()

In [ ]:
for i,year in enumerate(area_distribution['year'].unique()):
  data = area_distribution.loc[area_distribution['year'] == year, :]
  print("Number of unique admin regions {}".format(len(data['admin1'].unique())))
  print(sorted(data['admin1'].unique()))
  data.reset_index(drop=True, inplace=True)
  data.sort_values(by='priority_level_validated_by_the_clusters', inplace=True, ascending=False)
  plt.figure(figsize=(40,10))
  plt.bar(data['admin1'], data['priority_level_validated_by_the_clusters'])
  plt.xticks(rotation=45)
  plt.xlabel('admin1')
  plt.ylabel('Number of rows for {}'.format(year))
  plt.show()

In [ ]:
# dropping priority level validated by clusters & DS_ADMIN_2_Admnistratif_Burden_Data
df.drop(['DS_ADMIN_2_Admnistratif_Burden_Data'], axis=1, inplace=True)
df.drop(['priority_level_validated_by_the_clusters'], axis=1, inplace=True)
print(df.shape)

In [ ]:

categorical_columns = []
numerical_columns = []

for col in df.columns:
  if col != 'year':
    if df[col].dtype == object:
      categorical_columns.append(col)
    elif df[col].dtype == 'float64':
      numerical_columns.append(col)

In [ ]:
categorical_columns

In [ ]:
numerical_columns

In [ ]:
# encoding categorical columns to numerical columns
from sklearn.preprocessing import LabelEncoder

lbl = LabelEncoder()

for var in categorical_columns:
    df[var] = lbl.fit_transform(df[var])

print(df.shape)

In [ ]:
# exploring correlation amongst columns having numerical values 
corr = df[numerical_columns].corr()

fig, ax = plt.subplots()
fig.set_figheight(60)
fig.set_figwidth(60)
sns.heatmap(df.corr(method='pearson'), annot=True, fmt='.4f', 
            cmap=plt.get_cmap('coolwarm'), cbar=False, ax=ax)
ax.set_yticklabels(ax.get_yticklabels(), rotation="horizontal")
fig.savefig("out.png")

In [ ]:
# dropping features with correlation greater than 0.9 
corr = df[numerical_columns].corr().abs()

corr_cut_off = 0.9

upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))

to_drop = [column for column in upper.columns if any(upper[column] > corr_cut_off)] 

In [ ]:
to_drop

In [ ]:
df.drop(to_drop, axis=1, inplace=True)

In [ ]:
df.columns

In [ ]:
df['target'].value_counts()

In [ ]:
df.groupby('year')['target'].value_counts()

In [ ]:
# feature 'year' used for splitting the data into train test
target_column = 'target'

In [ ]:
df[target_column].hist()

In [ ]:
df.groupby('year')['target'].hist()

In [ ]:
# splitting the set into train and test set using the 'year'
train = df.loc[df['year'] != 2022, :].reset_index(drop=True)
test = df.loc[df['year'] == 2022, :].reset_index(drop=True)

In [ ]:
# removing the 'year' from the dataset (will lead to overfitting during training)
train.drop('year', axis=1, inplace=True)
test.drop('year', axis=1, inplace=True)

In [ ]:
xtrain = train.loc[:, train.columns != 'target']
ytrain = train['target']
xtest = test.loc[:, train.columns != 'target']
ytest = test['target']

In [ ]:
# qwk metric for calculating divergence regarding actual labels
def sklearn_qwk(y_true, y_pred) -> np.float64:
    """
    Function for measuring Quadratic Weighted Kappa with scikit-learn
    
    :param y_true: The ground truth labels
    :param y_pred: The predicted labels
    
    :return The Quadratic Weighted Kappa Score (QWK)
    """
    return cohen_kappa_score(y_true, y_pred, weights="quadratic")

In [ ]:
def one_away_accuracy(y_true, y_pred) -> np.float64:
    """
    Function for measuring 1-away-accuracy with scikit-learn
    
    :param y_true: The ground truth labels
    :param y_pred: The predicted labels
    
    :return 1_away_accuracy (1_accuracy)
    """
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)

    to_sum_list = []
    
    for i in range(cmx_data.shape[0]):
        to_sum_list.append(cmx_data[i][i])

    for i in range(1, cmx_data.shape[0]):
        to_sum_list.append(cmx_data[i][i - 1])
        to_sum_list.append(cmx_data[i-1][i])
    
    one_away_acc = sum(to_sum_list)/sum(np.concatenate(cmx_data))
    return one_away_acc

In [ ]:
kappa_scorer = make_scorer(sklearn_qwk)
one_acc_scorer = make_scorer(one_away_accuracy)

In [ ]:
# plotting confusion matrix
def print_cmx(y_true, y_pred, label):
    print("Confusion Matrix: {}".format(label))
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)

    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)

    plt.figure(figsize=(10, 7))
    sn.heatmap(df_cmx, annot=True)
    plt.show()

In [ ]:
model_outputs = []

####Random Forest####

In [ ]:
# random forest parameter optimization
# since KNN is a distance based algorithm we'll scale the features and then use imputation to fill the values.
# creating a pipeline for all the algorithms
pipe = Pipeline(steps=[
    ('scale', MinMaxScaler()),
    ('impute', KNNImputer(n_neighbors=3)),
    ('rf', RandomForestClassifier(random_state=2021, n_estimators=200))
])

# we have a list of possible values for each hyperparameter
params= { 
    'rf__max_features': ['auto', 'sqrt'],
    'rf__max_depth' : [4,5,6,8],
    'rf__min_samples_split': [2,5,10]
}


In [ ]:
feat = [x for x in xtrain.columns]

In [ ]:
# 10 fold cross validation is used for hyperparameter tuning; 
#Creating RF only on accuracy score as others (one_away and QWK) shouldn't affect much

clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf.fit(xtrain[feat], ytrain)

score = cross_val_score(clf.best_estimator_, xtrain[feat], ytrain, cv=10, scoring='accuracy')

# saving the best model for RandomForest for stacking
model_rf = clf.best_estimator_

model_rf.fit(xtrain[feat], ytrain)

pred = clf.best_estimator_.predict(xtest[feat])

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print(model_rf)

print_cmx(ytest, pred, "Accuracy as Metric")

# storing All 3 results for each algo
acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)


model_outputs.append({
    'model': "RandomForest",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})
model_outputs


####Plots: Accuracy and SHAP####

In [ ]:
# SHAP values are calculated for each class individually - Accuracy
explainer = shap.TreeExplainer(model_rf['rf'])
shap_values = np.array(explainer.shap_values(xtrain[feat]))


# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

In [ ]:
# code for drawing all dependency plots between features
for j in range(4):
  for i in range(len(feat)):
    print("****** Dependency plots for feature " + str(i) +" ******")
    shap.dependence_plot(i, shap_values[j], xtrain[feat]) 

####LightGBM####

In [ ]:
# lightgbm hyperparameter tuning
pipe = Pipeline(steps=[
    ('scale', MinMaxScaler()),
    ('impute', KNNImputer(n_neighbors=3)),
    ('lgb', lgb.LGBMClassifier(random_state=2021, n_estimators=200))
])

params = {'lgb__max_depth': [4, 6, 8],
                   'lgb__num_leaves': [16, 32],
                   'lgb__colsample_bytree': [0.65, 0.75, 0.9], 
                   'lgb__subsample': [0.7, 0.8, 0.9]}

In [ ]:
# Making two Lgb models; one which does parameter tuning on accuracy other on QWK

# 10 fold cross validation is used for hyperparameter tuning using accuracy

clf_acc = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf_acc.fit(xtrain[feat], ytrain)

score = cross_val_score(clf_acc.best_estimator_, xtrain[feat], ytrain, cv=10, scoring='accuracy')

model_lgb_acc = clf_acc.best_estimator_

model_lgb_acc.fit(xtrain[feat], ytrain)

pred = clf_acc.best_estimator_.predict(xtest[feat])

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "Accuracy as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "LightGBM_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

# 10 fold cross validation is used for hyperparameter tuning using QWK
clf_qwk = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

clf_qwk.fit(xtrain[feat], ytrain)

score_qwk = cross_val_score(clf_qwk.best_estimator_, xtrain[feat], ytrain, cv=10, scoring='accuracy')

model_lgb_qwk = clf_qwk.best_estimator_

model_lgb_qwk.fit(xtrain[feat], ytrain)

pred = clf_qwk.best_estimator_.predict(xtest[feat])

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "QWK as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "LightGBM_qwk",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

In [ ]:
model_outputs

####Another Step on LightGBM####

In [ ]:
# using an additional step to use lightgbm's inbuilt capacility to handle missing values
pipe = Pipeline(steps=[
    ('lgb', lgb.LGBMClassifier(random_state=2021, n_estimators=200))
])

params = {'lgb__max_depth': [4, 6, 8],
                   'lgb__num_leaves': [16, 32],
                   'lgb__colsample_bytree': [0.65, 0.75, 0.9], 
                   'lgb__subsample': [0.7, 0.8, 0.9]}

In [ ]:
# 10 fold cross validation is used for hyperparameter tuning with accuracy
clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf.fit(xtrain, ytrain)

score = cross_val_score(clf.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_lgb_noimp = clf.best_estimator_

model_lgb_noimp.fit(xtrain, ytrain)

pred = clf.best_estimator_.predict(xtest)

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "Accuracy as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "LightGBM_no_imputation_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

# 10fold cross validation is used for hyperparameter tuning with QWK
clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

clf.fit(xtrain, ytrain)

score_qwk = cross_val_score(clf.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_lgb_qwk_noimp = clf.best_estimator_

model_lgb_qwk_noimp.fit(xtrain, ytrain)

pred = clf.best_estimator_.predict(xtest)

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "QWK as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "LightGBM_no_imputation_qwk",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

In [ ]:
model_outputs

####Plots: Accuracy and SHAP####


In [ ]:
# SHAP values are calculated for each class individually
explainer = shap.TreeExplainer(model_lgb_noimp['lgb'])

shap_values = np.array(explainer.shap_values(xtrain))

# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

In [ ]:
for j in range(4):
  for i in range(len(feat)):
    print("****** Dependency plots for feature " + str(i) +" ******")
    shap.dependence_plot(i, shap_values[j], xtrain[feat]) 

In [ ]:
# normal Feature Importance: Choose a value of top k and dispaly top_k features of the model

top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf.best_estimator_.named_steps["lgb"].feature_importances_

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=importances.sort_values('feature_imp', ascending=False).head(top_k))

In [ ]:
importances

####XGBoost####

In [ ]:
# xgboost hyperparameter tuning
pipe = Pipeline(steps=[
    ('scale', MinMaxScaler()),
    ('impute', KNNImputer(n_neighbors=3)),
    ('xgb', xgb.XGBClassifier(random_state=2021, n_estimators=200, min_child_weight=3, use_label_encoder=True, eval_metric='mlogloss'))
])

params =  {
        'xgb__max_depth': [3, 5, 8],
        'xgb__subsample': [0.5, 0.7, 0.9],
        'xgb__colsample_bytree': [0.5, 0.7, 0.9],
    }

In [ ]:
ytrain.value_counts()

In [ ]:
# 10 fold cross validation is used for hyperparameter tuning with accuracy
clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf.fit(xtrain, ytrain)

score = cross_val_score(clf.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_xgb = clf.best_estimator_

model_xgb.fit(xtrain, ytrain)

pred = clf.best_estimator_.predict(xtest)

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "Accuracy as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "XGBoost_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

# 10fold cross validation is used for hyperparameter tuning with QWK
clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

clf.fit(xtrain, ytrain)

score_qwk = cross_val_score(clf.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_xgb_qwk = clf.best_estimator_

model_xgb_qwk.fit(xtrain, ytrain)

pred = clf.best_estimator_.predict(xtest)

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "QWK as Metric")

acc_qwk = accuracy_score(ytest, pred)

model_outputs.append({
    'model': "XGBoost_qwk",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})


In [ ]:
model_outputs

###Another Step for XGboost###

In [ ]:
# xgboost hyperparameter tuning
pipe = Pipeline(steps=[
  ('xgb', xgb.XGBClassifier(random_state=2021, n_estimators=200, min_child_weight=3, use_label_encoder=True, eval_metric='mlogloss'))
])

params =  {
        'xgb__max_depth': [3, 5, 8],
        'xgb__subsample': [0.5, 0.7, 0.9],
        'xgb__colsample_bytree': [0.5, 0.7, 0.9],
    }

In [ ]:
# 10 fold cross validation is used for hyperparameter tuning
clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf.fit(xtrain, ytrain)

score = cross_val_score(clf.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_xgb_noimp = clf.best_estimator_

model_xgb_noimp.fit(xtrain, ytrain)

pred = clf.best_estimator_.predict(xtest)

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "Accuracy as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "XGBoost_no_imputation_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

# 10fold cross validation is used for hyperparameter tuning
clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

clf.fit(xtrain, ytrain)

score_qwk = cross_val_score(clf.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_xgb_qwk_noimp = clf.best_estimator_

model_xgb_qwk_noimp.fit(xtrain, ytrain)

pred = clf.best_estimator_.predict(xtest)

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "QWK as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "XGBoost_no_imputation_qwk",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})



In [ ]:
model_outputs

####Plots: Accuracy and SHAP####


In [ ]:
# SHAP values are calculated for each class individually
explainer = shap.TreeExplainer(model_xgb_noimp['xgb'])

shap_values = np.array(explainer.shap_values(xtrain))

# feature importance for class i for loop
for i in range(4):
    print("****** Feature importances for class " + str(i) +" ******")
    shap.summary_plot(shap_values[i], xtrain[feat], plot_type="bar")
    shap.summary_plot(shap_values[i], features=xtrain[feat], feature_names=feat)

In [ ]:
# make dependency plots of all the features for class j
for j in range(4):
  for i in range(len(feat)):
    print("****** Dependency plots for feature " + str(i) +" ******")
    shap.dependence_plot(i, shap_values[j], xtrain[feat]) 

In [ ]:
#Normal Feature Importance: Choose a value of top k and dispaly top_k features of the model

top_k = 15

importances = pd.DataFrame()
importances ['feature'] = feat
importances ['feature_imp'] = clf.best_estimator_.named_steps["xgb"].feature_importances_

print("Feature_Imp")

plt.figure(figsize = (8, 12))
sns.barplot(x = 'feature_imp', y = 'feature', data=importances.sort_values('feature_imp', ascending=False).head(top_k))

In [ ]:
importances

####SVM####

In [ ]:
# for svm model we need to scale the numerical column - se define a pipeline
pipe = Pipeline(steps=[
    ('scale', MinMaxScaler()),
    ('impute', KNNImputer(n_neighbors=3)),
    ('svm', SVC(random_state=2021))
])

# defining parameter grid for finding the optimal C value using the training data
params = {'svm__C':[0.01, 10, 100, 1000, 2500, 5000]}

In [ ]:
# 10 fold cross validation is used for hyperparameter tuning
clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring='accuracy')

clf.fit(xtrain, ytrain)

score = cross_val_score(clf.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_svm = clf.best_estimator_

model_svm.fit(xtrain, ytrain)

pred = clf.best_estimator_.predict(xtest)

print("Classification Report for the test set with Accuracy as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "Accuracy as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "SVC_acc",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

# 10fold cross validation is used for hyperparameter tuning
clf = GridSearchCV(pipe, params, cv=10, n_jobs=-1, scoring=kappa_scorer)

clf.fit(xtrain, ytrain)

score_qwk = cross_val_score(clf.best_estimator_, xtrain, ytrain, cv=10, scoring='accuracy')

model_svm_qwk = clf.best_estimator_

model_svm_qwk.fit(xtrain, ytrain)

pred = clf.best_estimator_.predict(xtest)

print("Classification Report for the test set with QWK as metric {}".format(classification_report(ytest, pred)))

print_cmx(ytest, pred, "QWK as Metric")

acc = accuracy_score(ytest, pred)
qwk = sklearn_qwk(ytest, pred)
one_away_acc = one_away_accuracy(ytest, pred)

model_outputs.append({
    'model': "SVC_qwk",
    'cv_score_mean': score.mean(),
    'cv_score_std': score.std(),
    'test_accuracy': acc,
    'test_qwk': qwk,
    'test_one_away': one_away_acc
})

In [ ]:
model_outputs

####Summary Results of All Models####


In [ ]:
# Summary of performance of all the models

data_df = pd.DataFrame(model_outputs)

In [ ]:
data_df